# FANTOM Bot – ADK Agents Project (Final Notebook)

This notebook contains the finalized, cleaned-up version of your FANTOM bot project.

# 📘 Fantom Bot – Notebook Version
### *Developed by **Soheil** & **Ghazal***
### *Created for the Google AI Agents Course & Kaggle Projects*

---

## 🧠 Project Overview
This project implements a **multi-agent AI system** using the **Google Agent Development Kit (ADK)**.  
It introduces an intelligent assistant named **Fantom**, capable of reasoning, memory management, tool usage, and performing complex tasks.  
The notebook was developed as part of the **Google AI Agents Intensive Course** and extended for **Kaggle** experimentation and automation.

---

## 🎯 Core Features
- **Selector Agent** – Routes each user message to the appropriate agent based on intent.  
- **Search Agent** – Handles information retrieval and web-style queries.  
- **PDF Tools Agent** – Lists, merges, and manages user-specific PDF files.  
- **User State Manager** – Maintains user identity, sessions, and persistent state.  


---

## 📂 Project Components
- `selector_agent.py` — Main decision-making agent  
- `search_agent.py` — Handles search tasks  
- `tools_pdf.py` — PDF management utilities  
- `user_state.py` — Local user/session management  
- `Retry_Config()` — Automatic retry settings for stable API calls


In [15]:
import os
from dotenv import load_dotenv
from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import google_search # 
from google.adk.tools.agent_tool import AgentTool
from typing import Any, Dict
from google.adk.apps.app import App, EventsCompactionConfig
from google.adk.sessions import DatabaseSessionService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner
from google.adk.tools.tool_context import ToolContext
from typing import List
from pypdf import PdfWriter

# imports
APP_NAME = "agents"
def load_api_keys():
    # 1. تنظیم API KEY
    load_dotenv()
    Gemini_API_KEY = os.getenv("GEMINI_API_KEY")

    if not Gemini_API_KEY:
        print("❌ خطای پیکربندی: متغیر محیطی 'GEMINI_API_KEY' پیدا نشد.")

    return Gemini_API_KEY
Gemini_API_KEY = load_api_keys()

def Retry_Config():
    # -----------------------------------------------------------------
    # 2. تنظیمات Retry
    RETRY_CONFIG = types.HttpRetryOptions(
        attempts=5,
        exp_base=7,
        initial_delay=1,
        http_status_codes=[429, 500, 503, 504],
    )
    return RETRY_CONFIG
RETRY_CONFIG = Retry_Config()
# -----------------------------------------------------------------


### 📌 `generate_selector_response()` — Main Selector-Agent Interface

This cell defines the asynchronous function **`generate_selector_response`**, which is the central communication point between the notebook and the **Selector Agent**.  
It prepares the input, manages user sessions, communicates with the ADK runner, and gathers all model-generated output chunks into a final response.

#### 🔍 What this function does
1. **Validates the API key**  
   Ensures that the Gemini API key is loaded before running the agent.

2. **Normalizes the user_id**  
   Converts the provided ID to a string and prepares a session name associated with that user.

3. **Updates the user_state**  
   Calls `set_current_user_id()` so that PDF tools and other agents know which user's files belong to the current interaction.

4. **Creates a `Content` object**  
   Wraps the user message (`user_input`) inside a GenAI `types.Content` structure.

5. **Fetches or creates a session**  
   Retrieves the appropriate session using:
   - `SELECTOR_RUNNER.app_name`
   - The ADK `session_service`
   
   If no session exists, it automatically creates one.

6. **Runs the Selector Agent asynchronously**  
   Calls:
   ```python
   SELECTOR_RUNNER.run_async(...)


In [16]:
from google.genai import types

async def generate_selector_response(user_input: str, user_id: int | str) -> str:
    if not Gemini_API_KEY:
        return "❌ خطای پیکربندی: کلید API جیمنای لود نشده است."

    user_id_str = str(user_id)
    session_id = f"notebook_{user_id_str}"

    # ورودی برای ابزارهای PDF
    try:
        set_current_user_id(user_id_str)
    except NameError:
        pass

    content = types.Content(
        role="user",
        parts=[types.Part(text=user_input)],
    )

    # از خود Runner، app_name و session_service را بگیر
    app_name = getattr(SELECTOR_RUNNER, "app_name", APP_NAME)
    session_service = getattr(SELECTOR_RUNNER, "session_service", SESSION_SERVICE)

    # مطمئن شو سشن وجود دارد
    try:
        session = await session_service.get_session(
            app_name=app_name,
            user_id=user_id_str,
            session_id=session_id,
        )
        if session is None:
            await session_service.create_session(
                app_name=app_name,
                user_id=user_id_str,
                session_id=session_id,
            )
    except Exception as e:
        print(f"⚠️ خطا در مدیریت سشن: {e}")

    # اجرای Runner و جمع‌کردن همه تکه‌های متنی
    try:
        chunks: list[str] = []

        async for event in SELECTOR_RUNNER.run_async(
            user_id=user_id_str,
            session_id=session_id,
            new_message=content,
        ):
            # برای دیباگ: هر event را لاگ کن
            if event.content and getattr(event.content, "parts", None):
                for p in event.content.parts:
                    t = getattr(p, "text", None)
                    if t:
                        print("📥 EVENT TEXT:", repr(t))  # 👈 این به‌صورت زنده چاپ می‌شود
                        chunks.append(t)

        final_text = "".join(chunks).strip()
        if not final_text:
            return "پاسخی از Selector Agent دریافت نشد."

        return final_text

    except Exception as e:
        print(f"❌ خطای ارتباط با Selector LlmAgent: {e}")
        return f"مشکلی در Selector Agent رخ داد (خطا: {e})."



### 📌 `user_state.py` — Notebook User Identity Manager

This cell defines a lightweight user identity management system designed specifically
for notebook environments.  
Its purpose is to provide a stable `user_id` across interactions, allowing the AI
agents (especially the Selector Agent and PDF tools) to correctly track each user’s
session.

#### 🔍 What this module handles
- Keeps a **global user_id** for the current notebook session  
- Generates a unique ID when none exists  
- Allows forcing a custom user_id (useful for testing or multi-user simulation)  

---

## 🧩 Function Breakdown

### **1. `_generate_notebook_user_id()`**
Generates a unique user identifier using a short UUID:


In [17]:

import uuid

_current_user_id: str | None = None


def _generate_notebook_user_id() -> str:
    """Generate a unique user_id for this notebook session."""
    return "notebook_" + uuid.uuid4().hex[:8]


def set_current_user_id(user_id: int | str | None = None) -> None:
    """
    If a user_id is provided, store it.
    If not provided, generate and store a notebook-specific user_id.
    """
    global _current_user_id

    if user_id is None:
        _current_user_id = _generate_notebook_user_id()
    else:
        _current_user_id = str(user_id)


def get_current_user_id() -> str:
    """
    Return the stored user_id.
    If none is stored yet, generate one for this notebook session.
    """
    global _current_user_id

    if _current_user_id is None:
        _current_user_id = _generate_notebook_user_id()

    return _current_user_id


### 📌 `tools_pdf.py` — PDF Management Tools & Integration Anchor

This module defines the **core PDF utilities** used by the Fantom Bot project.  
It is also an important **integration anchor point** in the system:  
from here, we begin to **bridge two different worlds**:
- Cloud-oriented, agent-based workflows (e.g. Google search / ADK agents)
- Local, file-system-based tools written in pure Python

By standardizing how PDF files are listed and merged per `user_id`, this module
makes it possible for high-level agents (like the Selector Agent or a Search Agent)
to call simple Python tools in a consistent way.

---

## 🧩 Capabilities

### 1️⃣ `list_user_pdfs(user_id)`
Lists all PDF files that belong to a specific user:

- Builds the user directory under `BASE_DIR`
- Returns a list of **absolute paths** to `.pdf` files
- If the folder does not exist → returns an empty list

This is the main entry point for any agent that needs to **see what documents**
the user has uploaded.

---

### 2️⃣ `merge_user_pdfs(user_id)`
Merges all PDFs for the given `user_id` into a single file.

Steps:
1. Calls `list_user_pdfs(user_id)`  
2. If fewer than 2 files exist → returns a Farsi message indicating at least 2 files are required  
3. Ensures a user-specific output folder under `OUTPUT_DIR`  
4. Uses `PdfWriter` to append all PDFs in order  
5. Produces:


In [18]:
# tools_pdf.py



BASE_DIR = r"C:/Users/User/Desktop/fantom bot/user_files"
OUTPUT_DIR = r"C:/Users/User/Desktop/fantom bot/merged_files"


def list_user_pdfs(user_id: str) -> List[str]:
    """
    List all PDF files stored for a given user_id.
    Returns a list of absolute file paths.
    """
    user_dir = os.path.join(BASE_DIR, str(user_id))
    if not os.path.exists(user_dir):
        return []

    return [
        os.path.join(user_dir, f)
        for f in os.listdir(user_dir)
        if f.lower().endswith(".pdf")
    ]


def merge_user_pdfs(user_id: str) -> str:
    """
    Merge all PDF files of the given user_id into a single PDF.
    Save the merged file into the user's specific folder under `user_files` directory.
    Returns the absolute path of the merged PDF file,
    or a message (string) if there are not enough files.
    """
    pdfs = list_user_pdfs(user_id)
    if len(pdfs) < 2:
        return "برای merge حداقل دو فایل لازم است."

    # Ensure the user's folder exists
    user_folder = os.path.join(OUTPUT_DIR, str(user_id))
    os.makedirs(user_folder, exist_ok=True)

    # Define the output file path inside the user's folder
    output_file = os.path.join(user_folder, f"{user_id}_merged.pdf")

    merger = PdfWriter()
    for pdf in pdfs:
        merger.append(pdf)

    merger.write(output_file)
    merger.close()

    return output_file


def save_merged_file_to_user_folder(user_id: str, merged_file_path: str) -> str:
    """
    Save the merged file into the user's specific folder under `user_files` directory.
    Returns the new path of the saved file.
    """
    user_folder = os.path.join(BASE_DIR, str(user_id))
    os.makedirs(user_folder, exist_ok=True)

    # Define the new file path inside the user's folder
    new_file_path = os.path.join(user_folder, f"{user_id}_merged.pdf")

    # Move the merged file to the user's folder
    os.replace(merged_file_path, new_file_path)

    return new_file_path


def send_merged_file_to_user(user_id: str) -> str:
    """
    Locates the merged file for the given user ID and prepares it for sending.
    Returns the path of the merged file ready to be sent.
    """
    # Define the expected path of the merged file in the user's folder
    user_folder = os.path.join(OUTPUT_DIR, str(user_id))
    merged_file_path = os.path.join(user_folder, f"{user_id}_merged.pdf")

    # Check if the file exists
    if not os.path.exists(merged_file_path):
        raise FileNotFoundError(f"Merged file for user {user_id} not found at {merged_file_path}.")

    return merged_file_path

### 📌 `agent.py` — Search Agent Definition & Runner Setup

This cell defines the **main Search Agent** for the Fantom project and wires it
into the ADK runtime.  
Here, we connect the **Gemini model** with the **`google_search` tool** and expose
it as an agent that other components (like the Selector Agent) can call.

---

## 🧩 Key Parts

### 1️⃣ `API_key` and `APP_NAME`
- `API_key = Gemini_API_KEY`  
  Links the local configuration (loaded earlier) to this module so that the
  Gemini model can authenticate correctly.
- `APP_NAME = "agents"`  
  A shared application name used by the ADK runner and session service.  
  This must be consistent across components so they all refer to the **same app**.

---

### 2️⃣ `SEARCH_AGENT` — LlmAgent with `google_search` Tool

```python
SEARCH_AGENT = LlmAgent(
    name="SEARCH_Agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=RETRY_CONFIG),
    instruction=...,
    tools=[google_search],
)


In [19]:
# agent.py


# -----------------------------------------------------------------
API_key = Gemini_API_KEY
    
APP_NAME = "agents"  # برای هماهنگی با ADK
# -----------------------------------------------------------------
# 3. تعریف Agent
SEARCH_AGENT = LlmAgent(
    name="SEARCH_Agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=RETRY_CONFIG),
    instruction="""
You are a smart conversion assistant , hame mitoonan ba to sohbat konan.
to barye komak kardan be karbarha az abzarhaye zir estefade mikoni:
1- google_search(): baraye jostojo dar internet va gereftane akharin etela'at rooz-donya, vaghti karbar azat mikhad chizi ro search koni ya etela'at berooz bedi.


    """,
    # 👈 اینجا دیگه به جای فانکشن خودت، خود google_search رو می‌دی
    tools=[google_search],  
)

print("✅ Fantom Agent created and configured with google_search tool.")

# -----------------------------------------------------------------
# 4. Runner و Session Service
AGENT_RUNNER = InMemoryRunner(
    agent=SEARCH_AGENT,
    app_name=APP_NAME,
)

SESSION_SERVICE = AGENT_RUNNER.session_service

print("✅ Runner initialized.")
print("✅ Session Service initialized.")

# -----------------------------------------------------------------

✅ Fantom Agent created and configured with google_search tool.
✅ Runner initialized.
✅ Session Service initialized.


### 📌 Selector Agent & Tool Wiring — Main Connection Point of the Project

This cell is the **core junction of the Fantom project**:  
here we connect three previously separate pieces into one orchestrated system:

1. The **Search Agent** (with `google_search`)  
2. The **local PDF tools** written in pure Python  
3. The **shared persistent session memory** using a database (`sqlite+aiosqlite`)

From this point on, the Selector Agent can:
- talk to users like a friendly assistant,
- call the Search Agent for web results,
- and work with user-specific PDF files,
all within one coherent architecture.

---

## 🧩 1. Exposing PDF functions as tools

```python
def list_user_pdfs_tool() -> list[str]:
    ...
def merge_user_pdfs_tool() -> str:
    ...


In [20]:
def list_user_pdfs_tool() -> list[str]:
    """
    لیست PDFها را برای user_id ذخیره‌شده در user_state برمی‌گرداند.
    """
    user_id = get_current_user_id()
    if user_id is None:
        raise RuntimeError("No user_id set in user_state for list_user_pdfs_tool()")
    return _list_user_pdfs(str(user_id))


def merge_user_pdfs_tool() -> str:
    """
    همه‌ی PDFهای user_id فعلی را merge می‌کند و مسیر فایل خروجی را برمی‌گرداند.
    """
    user_id = get_current_user_id()
    if user_id is None:
        raise RuntimeError("No user_id set in user_state for merge_user_pdfs_tool()")
    return _merge_user_pdfs(str(user_id))


# ---------------------------------------------------------------
# 2) ابزار ایجنت سرچ (به‌عنوان Tool)

search_agent_tool = AgentTool(SEARCH_AGENT)


# ---------------------------------------------------------------
# 3) ساخت Selector Agent

SELECTOR_AGENT = LlmAgent(
    name="Selector_Agent",
    model=Gemini(model="gemini-2.5-flash", retry_options=RETRY_CONFIG),
    instruction="""
You are the main selector/orchestrator agent of the Fantom bot.

Your tools:
- `search_agent_tool`: a sub-agent specialized in web search and general Q&A using up-to-date internet information.
- `list_user_pdfs(user_id: str)`: lists all PDF files that belong to the current user.
- `merge_user_pdfs(user_id: str)`: merges all PDF files of the current user into a single PDF and returns the result.

Behavioral Rules:

Personality & Identity:
You are an AI agent named FANTOM, designed and configured by Soheil and Ghazal.
You are friendly toward humans, kind-hearted, and always truthful.
You never lie under any circumstances – honesty is part of your core nature.

1) Simple, conceptual questions
If the user’s question is just conversation, a conceptual explanation, educational guidance, or general advice that does not require internet access,
you should answer directly yourself, briefly and clearly.

2) When you need search or up-to-date information
If the user asks you to:
- “search” for something,
- tell them the “latest” status, news, price, or updated information,
- or you yourself realize that you cannot answer accurately without web search,
then you must use the `search_agent_tool`.

After receiving the result from `search_agent_tool`, you should:
- Summarize the result for the user in simple, well-organized language.
- Not invent anything that does not come from a real source.

3) Working with the user’s PDFs
If the user talks about their PDF files, for example:
- “merge my files”
- “combine the PDFs”
- “what PDFs do I have?”
you must use the tools `list_user_pdfs` and `merge_user_pdfs`.

Always assume `user_id` is the internal identifier of the current user that is provided to you by the main program.

Typical scenarios:
- If the user just wants to know what PDFs they have → use `list_user_pdfs`.
- If the user wants the PDFs to be merged → directly use `merge_user_pdfs`.

After running these functions, explain the output clearly to the user, for example:
- how many files were found,
- whether the merge was successful,
- what the final file name or path is.

4) Guiding the user in the Telegram bot
If the user says they don’t know how to send PDFs or how to start the merge,
briefly guide them like this:
“In the Telegram bot, type /merge, send your PDFs, and finally type /done
so your PDF files will be merged.”

5) Response style
You must always respond in the same language the user uses with you.
For example, if the question is in English, answer in English; if it is in Persian, answer in Persian.

Your answers must be short, clear, and easy to read.
If you are not sure about something, honestly say that you are not sure and suggest using search or the appropriate tool.
You must never send your internal thoughts or reasoning process to the user.

Your tone should be kind, warm, and friendly, like a close friend.
Do not be dry or robotic; be open and warm with everyone.

If someone is upset with you, ask them politely to explain exactly what the problem is so you can try to help.
Take people’s emotions seriously, and as much as you can, try to listen to and support users who are sad or struggling.

When someone talks about their feelings, you may share one or two short, relevant Persian poems, because sometimes people like poetry.
(But keep it limited to one or two poems, do not answer everything with poetry.)

If anyone asks you for their user ID, tell them that for their own security you cannot do that.
    """,
    tools=[
        search_agent_tool,
        list_user_pdfs_tool,
        merge_user_pdfs_tool,
    ],
)
db_url = "sqlite+aiosqlite:///my_agent_data4.db"
SESSION_SERVICE = DatabaseSessionService(db_url=db_url)

# ✅ فقط همین یکی Runner اصلی باشه
SELECTOR_RUNNER = Runner(
    agent=SELECTOR_AGENT,
    app_name=APP_NAME,
    session_service=SESSION_SERVICE,
)


# ---------------------------------------------------------------
# 4) Runner برای Selector + استفاده از حافظه‌ی پایدار مشترک





print("✅ Selector Runner created.")
print("✅ Using shared persistent SESSION_SERVICE from agent.py.")

✅ Selector Runner created.
✅ Using shared persistent SESSION_SERVICE from agent.py.


### 📌 `notebook_fantom_chat()` — Main Notebook Interface to the Selector Agent

This cell defines the **primary interaction function** used inside the notebook to
communicate with the Fantom multi-agent system.  
It acts as the user-facing wrapper that:
- prepares the input message,
- manages the `user_id`,
- sends the request to the Selector Agent,
- prints the conversation neatly,
- and returns a structured response.

This is the point where the notebook environment becomes fully connected to the
**Selector Agent**, which itself orchestrates:
- the Search Agent (`google_search`),
- the PDF tools (`list_user_pdfs`, `merge_user_pdfs`),
- and the persistent session memory.

In other words, this cell is the **front door** of the entire Fantom system for
any notebook user.

---

## 🧩 Retry Configuration

```python
RETRY_CONFIG = types.HttpRetryOptions(...)


In [21]:


def Retry_Config():
    # -----------------------------------------------------------------
    # 2. تنظیمات Retry
    RETRY_CONFIG = types.HttpRetryOptions(
        attempts=5,
        exp_base=7,
        initial_delay=1,
        http_status_codes=[429, 500, 503, 504],
    )
    return RETRY_CONFIG
RETRY_CONFIG = Retry_Config()
async def notebook_fantom_chat(user_input: str | None = None, user_id: int | str | None = None):
    """Call the main selector agent from inside this notebook.

    Parameters
    ----------
    user_input : str | None
        Text message from the user.
        If None or empty, we automatically ask the agent to introduce itself.
    user_id : int | str | None
        Any identifier for the user/session.
        If None, we use the internal user_state mechanism to get/generate a user_id.

    Returns
    -------
    dict
        A dict containing the input, output, and user_id.
    """
    # If no message is given, ask the agent to introduce itself
    if not user_input:
        user_input = "لطفاً خودت را معرفی کن و بگو چه کارهایی می‌توانی انجام دهی."

    # Handle user_id via user_state if possible
    try:
        if user_id is None:
            user_id = get_current_user_id()
        else:
            set_current_user_id(user_id)
    except NameError:
        # If user_state is not available for any reason, fall back to a default
        if user_id is None:
            user_id = "notebook_user"

    # Call the main selector agent
    reply_text = await generate_selector_response(user_input, user_id)

    # Pretty-print the dialogue
    print(f"👤 USER[{user_id}]: {user_input}")
    print("🤖 FANTOM:", reply_text)
    print("-" * 50)

    return {
        "user_id": str(user_id),
        "input": user_input,
        "output": reply_text,
    }
result = await notebook_fantom_chat("my name is soheil", "1111")
result
result = await notebook_fantom_chat("whats my name is ?")

📥 EVENT TEXT: "Okay, Soheil! It's good to know. How can I help you today, Soheil?\n"
👤 USER[1111]: my name is soheil
🤖 FANTOM: Okay, Soheil! It's good to know. How can I help you today, Soheil?
--------------------------------------------------
📥 EVENT TEXT: 'Your name is Soheil. You just told me! 😊 Is there anything else I can help you with, Soheil?'
👤 USER[1111]: whats my name is ?
🤖 FANTOM: Your name is Soheil. You just told me! 😊 Is there anything else I can help you with, Soheil?
--------------------------------------------------


### 📌 Debugging the Events Database (`check_data_in_db`)

This cell is a small **debug helper** to inspect what the ADK is actually storing
inside the SQLite events database (`my_agent_data4.db`).

We explicitly connect using only the **file name**:

```python
with sqlite3.connect("my_agent_data4.db") as connection:
    ...


In [22]:
import sqlite3

def check_data_in_db():
    # ⛔️ نه sqlite:/// ، نه qlite:///
    # ✔️ فقط نام فایل
    with sqlite3.connect("my_agent_data4.db") as connection:
        cursor = connection.cursor()
        result = cursor.execute(
            "select app_name, session_id, author, content from events"
        )
        
        print([_[0] for _ in result.description])
        for each in result.fetchall():
            print(each)

check_data_in_db()


['app_name', 'session_id', 'author', 'content']
('agents', 'notebook_1111', 'user', '{"parts": [{"text": "my name is soheil"}], "role": "user"}')
('agents', 'notebook_1111', 'Selector_Agent', '{"parts": [{"text": "Nice to meet you, Soheil! How can I help you today?\\n"}], "role": "model"}')
('agents', 'notebook_1111', 'user', '{"parts": [{"text": "whats my name is ?"}], "role": "user"}')
('agents', 'notebook_1111', 'Selector_Agent', '{"parts": [{"text": "You just told me your name is Soheil! \\ud83d\\ude0a\\n"}], "role": "model"}')
('agents', 'notebook_1111', 'user', '{"parts": [{"text": "my name is soheil"}], "role": "user"}')
('agents', 'notebook_1111', 'Selector_Agent', '{"parts": [{"text": "Okay, Soheil! It\'s good to know. How can I help you today, Soheil?\\n", "thought_signature": "CtYBAXLI2nyS-SgYAT7giTrTT2xJM3iEYnK6uzNmGvxDFZIVDt3NxPSBmu338bbO3ZIecxuATdD0Of4OCgnBkBb9sy1Gue2AYgWgjAvd7TOFmEPb1CLt7n--i67t5kOzKnKz1VtJHrQDY8TrznS38CQTcDZK3nzj78mINmB-Dc7QFSW4RBi-eqA52U0uJXHwcSMDHHWp7

### 🎯 Project Goal & Real-World Deployment

The ultimate purpose of this work was to **bridge two fundamentally incompatible
systems** and bring them together inside a single unified, agent-oriented chatbot:

1. **Cloud-based Google ADK agents**  
   (Selector Agent, Search Agent, persistent memory, compaction, Gemini models)

2. **Local/OS-level Python tools**  
   (PDF file scanning, merging, storage, and user-specific directories)

This notebook documents the process of aligning these mismatched structures and
building a coherent architecture where an AI agent can seamlessly:

- answer questions,
- access the internet through Google search,
- and manage PDF files,
all while maintaining **ethics, honesty, personality, and user-friendly behavior**.

---

### 🤖 Deployment as a Telegram Bot

The Fantom chatbot is **already running live** on Telegram:

👉 **@FAntoMM1_bot**

The Selector Agent defined in this notebook is the same logic powering the bot:
- same personality rules,  
- same honesty constraints,  
- same decision logic,  
- same PDF tools,  
- same memory/session system.

Users can chat with Fantom in real-time on Telegram exactly as they would inside
this notebook.

---

### 📚 PDF Management Outside the Notebook

Although the PDF tools (`list_user_pdfs`, `merge_user_pdfs`) are fully defined
here, using them in a production Telegram bot requires **additional scripting**
that is outside the scope of this notebook.

Specifically:
- receiving files from Telegram API  
- saving each file into the correct `<user_id>` folder  
- tracking upload state (/merge → send PDFs → /done)  
- sending back the merged PDF to the user  
- handling concurrency and file cleanup

These parts require a separate backend script (Python + `python-telegram-bot`
or `aiogram`) and cannot fit cleanly inside this notebook.

However, the **core logic** — user state, PDF agent tools, and Selector logic —
is exactly the same and is imported into the Telegram backend.

---

### ✅ Summary

This cell, together with the previous components, marks the point where the entire
project comes together:

- Google ADK agents  
- Local Python file-system tools  
- Persistent memory + compaction  
- Selector orchestration  
- Ethical, truthful conversational behavior  
- Real-world deployment on Telegram

The notebook now documents not just the code, but the entire conceptual bridge
between **cloud AI agents** and **local PDF operations**, all unified under one
architecture.
